###**Pressure datalogger correction for barometric pressure fluctuations**

At the moment, this is set up for Onset HOBO and baro files.

In [ ]:
#Connect to Drive
from google.colab import drive
drive.mount("/content/drive/")
import pandas as pd
import os

Mounted at /content/drive/


Specify names for LTC and barometric files. Path name should be the same- make sure working files are in the preprocess_files folder.

In [ ]:
path='/content/drive/My Drive/CZN_hydro/Water/preprocess_files/'
datfile='BN_A_20190314_LT.csv'
barofile='BN_20190314_Baro.csv'

In [ ]:
#Open data logger file, skip first row, only first 4 columns
df=pd.read_csv(path+datfile,skiprows=1,usecols = [i for i in range(4)])
df.rename(columns = {list(df)[1]:'DateTime',list(df)[2]:'LOGGER_P_kPa',
                     list(df)[3]:'LOGGER_Temp_C'}, inplace = True) 
df['DateTime']=pd.to_datetime(df['DateTime'])
df=df.set_index('DateTime')
df=df.drop(['#'],axis=1)
#Drop any rows at the end with no value
s=str(df['LOGGER_P_kPa'].last_valid_index())
df=df[df.index<=s]

#Bring in barometric data
barodf=pd.read_csv(path+barofile,skiprows=1,usecols = [i for i in range(4)])
barodf.rename(columns = {list(barodf)[1]:'DateTime',list(barodf)[2]:'BARO_P_kPa',
                         list(barodf)[3]:'BARO_Temp_C'}, inplace = True) 
barodf['DateTime']=pd.to_datetime(barodf['DateTime'])
barodf=barodf.set_index('DateTime')
barodf=barodf.drop(['#'],axis=1)

#Merge two dataframes, keeps ALL LTC records and interpolates any missing baro P values
df=df.merge(barodf['BARO_P_kPa'],how='left',left_index=True,right_index=True)
df['BARO_P_kPa']=df['BARO_P_kPa'].interpolate(axis=0)

In [ ]:
te=df['LOGGER_Temp_C']
p_lbft3=(((999.83952+16.945176*te)-(7.9870401e-03*(te**2))-(46.170461e-06*(te**3))+
      (105.56302e-09*(te**4)) - (280.54253e-12*(te**5))) / (1 + 16.879850e-03*te))*0.0624279606
#Conversion constants
PSI_PSF=144
KPA_PSI=0.1450377
FT_M=0.3048
PSI_FT=2.0366

df.head()

,LOGGER_Pressure_kPa,LOGGER_Temp_C,BARO_Pressure_kPa
DateTime,,,
2019-03-14 19:00:00,114.620,9.077,101.745
2019-03-14 19:15:00,114.591,8.779,101.765
2019-03-14 19:30:00,114.553,8.779,101.714
2019-03-14 19:45:00,114.543,8.779,101.680
2019-03-14 20:00:00,114.591,8.779,101.699


In [ ]:
#Convert baro pressure to meters water
df.rename(columns = {'BARO_P_kPa':'BARO_mWater'}, inplace=True)
df['BARO_mWater']=(df['BARO_mWater']*KPA_PSI*PSI_FT*FT_M)
BP1=df['BARO_mWater'][0]

#Convert logger pressure to meters water with density compensation
df['LOGGER_mWater_corr']=((KPA_PSI*PSI_PSF*df['LOGGER_P_kPa']/p_lbft3)*FT_M)+(BP1-df['BARO_mWater'])

df.head()

,LOGGER_Pressure_kPa,LOGGER_Temp_C,BARO_mWater,LOGGER_mWater_corr
DateTime,,,,
2019-03-14 19:00:00,114.620,9.077,9.160405,11.690614
2019-03-14 19:15:00,114.591,8.779,9.162206,11.685600
2019-03-14 19:30:00,114.553,8.779,9.157614,11.686316
2019-03-14 19:45:00,114.543,8.779,9.154553,11.688357
2019-03-14 20:00:00,114.591,8.779,9.156264,11.691542


In [ ]:
#Save a copy to the folder with a suffix and delete old copy
df.to_csv(path+datfile.split('.')[0]+'_baroCorrect.csv')
os.remove(path+datfile)